### Imports

In [136]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.patches import Patch
import os, shutil
import torch
import torch.nn.functional as F

#from sklearn.model_selection import train_test_split
from pathlib import Path
from torch import nn
from torch.utils.data.dataset import Dataset
from torchvision import models, transforms
from collections import defaultdict
from random import shuffle
from PIL import Image
from shapely.geometry import Polygon
from tqdm import tqdm_notebook


%matplotlib inline

In [137]:
class Line:
    # ax + by + c = 0
    def __init__(self, tensor1, tensor2):
        self.a = tensor2[1] - tensor1[1] #v2.y - v1.y 
        self.b = tensor1[0] - tensor2[0]#v1.x - v2.x
        self.c = torch.det(torch.stack((tensor2, tensor1)))#v2.cross(v1)

    def __call__(self, p):
        return self.a * p[0] + self.b * p[1] + self.c #self.a*p.x + self.b*p.y + self.c

    def intersection(self, other):
        # See e.g.     https://en.wikipedia.org/wiki/Line%E2%80%93line_intersection#Using_homogeneous_coordinates
        if not isinstance(other, Line):
            return NotImplemented
        w = self.a * other.b - self.b * other.a
        return torch.FloatTensor([
            (self.b * other.c - self.c * other.b) / w,
            (self.c * other.a - self.a * other.c) / w
        ])


def rectangle_vertices(cx, cy, w, h, a):
    radian = np.pi * a.double() / 180
    dx = w/2
    dy = h/2
    dxcos = dx * torch.cos(radian)
    dxsin = dx * torch.sin(radian)
    dycos = dy * torch.cos(radian)
    dysin = dy * torch.sin(radian)
    return (
        torch.FloatTensor([cx, cy]) + torch.FloatTensor([-dxcos - -dysin, -dxsin + -dycos]),
        torch.FloatTensor([cx, cy]) + torch.FloatTensor([dxcos - -dysin,  dxsin + -dycos]),
        torch.FloatTensor([cx, cy]) + torch.FloatTensor([dxcos -  dysin,  dxsin +  dycos]),
        torch.FloatTensor([cx, cy]) + torch.FloatTensor([-dxcos -  dysin, -dxsin +  dycos])
    )

def intersection_area(r1, r2):
    # r1 and r2 are in (center_x, center_y, width, height, rotation) representation
    # First convert these into a sequence of vertices

    rect1 = rectangle_vertices(r1[0], r1[1], r1[3], r1[2], r1[4])
    rect2 = rectangle_vertices(r2[0], r2[1], r2[3], r2[2], r2[4])

    # Use the vertices of the first rectangle as
    # starting vertices of the intersection polygon.
    intersection = rect1

    # Loop over the edges of the second rectangle
    for p, q in zip(rect2, rect2[1:] + rect2[:1]):
        if len(intersection) <= 2:
            break # No intersection

        line = Line(p, q)

        # Any point p with line(p) <= 0 is on the "inside" (or on the boundary),
        # any point p with line(p) > 0 is on the "outside".

        # Loop over the edges of the intersection polygon,
        # and determine which part is inside and which is outside.
        new_intersection = []
        line_values = [line(t) for t in intersection]
        for s, t, s_value, t_value in zip(
            intersection, intersection[1:] + intersection[:1],
            line_values, line_values[1:] + line_values[:1]):
            if s_value <= 0:
                new_intersection.append(s)
            if s_value * t_value < 0:
                # Points are on opposite sides.
                # Add the intersection of the lines to new_intersection.
                intersection_point = line.intersection(Line(s, t))
                new_intersection.append(intersection_point)

        intersection = new_intersection

    # Calculate area
    if len(intersection) <= 2:
        return 0

    #return 0.5 * sum(p.x*q.y - p.y*q.x for p, q in
    #                 zip(intersection, intersection[1:] + intersection[:1]))
    return 0.5 * sum(p[0] * q[1] - p[1] * q[0] for p, q in
                     zip(intersection, intersection[1:] + intersection[:1]))


def find_jaccard_overlap(predicted_prior_set, true_prior_set):
    intersection = torch.zeros(len(predicted_prior_set), len(true_prior_set))
    area1 = torch.zeros(predicted_prior_set.shape[0])
    area2 = torch.zeros(true_prior_set.shape[0])
        
    for i, box1 in tqdm_notebook(enumerate(predicted_prior_set), total=len(predicted_prior_set)):
        for j, box2 in tqdm_notebook(enumerate(true_prior_set), total=len(true_prior_set)):
            intersection[i][j] = intersection_area(box1, box2)
            
            if i == 0:
                area2[j] = box2[3] * box2[2]
        area1[i] = box1[3] * box1[2]
    
    union = area1.unsqueeze(1) + area2.unsqueeze(0) - intersection
    return intersection / union 

### Modifying SSD300.detect_objects

In [138]:
class SSD300_TEST(nn.Module):
    """
    The SSD300 network - encapsulates the base VGG network, auxiliary, and prediction convolutions.
    """

    def __init__(self, n_classes):
        super(SSD300_TEST, self).__init__()

        self.n_classes = n_classes # 2?

        #self.base = VGGBase()
        #self.aux_convs = AuxiliaryConvolutions()
        #self.pred_convs = PredictionConvolutions(n_classes)

        # Since lower level features (conv4_3_feats) have considerably larger scales, we take the L2 norm and rescale
        # Rescale factor is initially set at 20, but is learned for each channel during back-prop
        self.rescale_factors = nn.Parameter(torch.FloatTensor(1, 512, 1, 1))  # there are 512 channels in conv4_3_feats
        nn.init.constant_(self.rescale_factors, 20)

        # Prior boxes
        self.priors_cxcy = self.create_prior_boxes() #

    def forward(self, image):
        """
        Forward propagation.
        :param image: images, a tensor of dimensions (N, 3, 300, 300)
        :return: 8732 locations and class scores (i.e. w.r.t each prior box) for each image
        """
#         # Run VGG base network convolutions (lower level feature map generators)
#         conv4_3_feats, conv7_feats = self.base(image)  # (N, 512, 38, 38), (N, 1024, 19, 19)

#         # Rescale conv4_3 after L2 norm
#         norm = conv4_3_feats.pow(2).sum(dim=1, keepdim=True).sqrt()  # (N, 1, 38, 38)
#         conv4_3_feats = conv4_3_feats / norm  # (N, 512, 38, 38)
#         conv4_3_feats = conv4_3_feats * self.rescale_factors  # (N, 512, 38, 38)
#         # (PyTorch autobroadcasts singleton dimensions during arithmetic)

#         # Run auxiliary convolutions (higher level feature map generators)
#         conv8_2_feats, conv9_2_feats, conv10_2_feats, conv11_2_feats = \
#             self.aux_convs(conv7_feats)  # (N, 512, 10, 10),  (N, 256, 5, 5), (N, 256, 3, 3), (N, 256, 1, 1)

#         # Run prediction convolutions (predict offsets w.r.t prior-boxes and classes in each resulting localization box)
#         locs, classes_scores = self.pred_convs(conv4_3_feats, conv7_feats, conv8_2_feats, conv9_2_feats, conv10_2_feats,
#                                                conv11_2_feats)  # (N, 8732, 4), (N, 8732, n_classes)

#         return locs, classes_scores

    def create_prior_boxes(self):
        """
        Create the 8732 prior (default) boxes for the SSD300, as defined in the paper.
        :return: prior boxes in center-size coordinates, a tensor of dimensions (8732, 4)
        """
        fmap_dims = {'conv4_3': 38,
                     'conv7': 19,
                     'conv8_2': 10,
                     'conv9_2': 5,
                     'conv10_2': 3,
                     'conv11_2': 1}

        obj_scales = {'conv4_3': 0.006,
                      'conv7': 0.0728,
                      'conv8_2': 0.1396,
                      'conv9_2': 0.2064,
                      'conv10_2': 0.2732,
                      'conv11_2': 0.34}

        aspect_ratios = {'conv4_3': [0.5, 0.75, 1., 1.25, 1.75],
                         'conv7': [0.5, 0.75, 1., 1.25, 1.75, 3.],
                         'conv8_2': [0.5, 0.75, 1., 1.25, 1.75, 3.],
                         'conv9_2': [0.5, 0.75, 1., 1.25, 1.75, 3.],
                         'conv10_2': [0.5, 0.75, 1., 1.25, 1.75],
                         'conv11_2': [0.5, 0.75, 1., 1.25, 1.75]}
        
        rotation_angles = {0, -45, -90}

        fmaps = list(fmap_dims.keys())

        prior_boxes = []

        for k, fmap in enumerate(fmaps):
            for i in range(fmap_dims[fmap]):
                for j in range(fmap_dims[fmap]):
                    cx = (j + 0.5) / fmap_dims[fmap]
                    cy = (i + 0.5) / fmap_dims[fmap]

                    for ratio in aspect_ratios[fmap]:
                        for angle in rotation_angles:
                            prior_boxes.append([cx, 
                                                cy, 
                                                obj_scales[fmap] * np.sqrt(ratio), 
                                                obj_scales[fmap] / np.sqrt(ratio), 
                                                angle])
                        
                        """
                        # For an aspect ratio of 1, use an additional prior whose scale is the geometric mean of the
                        # scale of the current feature map and the scale of the next feature map
                        if ratio == 1.:
                            try:
                                additional_scale = sqrt(obj_scales[fmap] * obj_scales[fmaps[k + 1]])
                            # For the last feature map, there is no "next" feature map
                            except IndexError:
                                additional_scale = 1.
                            prior_boxes.append([cx, cy, additional_scale, additional_scale])
                        """

        prior_boxes = torch.FloatTensor(prior_boxes).cuda()  # (8732, 4)
        prior_boxes.clamp_(0, 1)  # (8732, 4)

        return prior_boxes

    def detect_objects(self, predicted_locs, predicted_scores, min_score, max_overlap, top_k):
        """
        Decipher the 8732 locations and class scores (output of ths SSD300) to detect objects.
        For each class, perform Non-Maximum Suppression (NMS) on boxes that are above a minimum threshold.
        :param predicted_locs: predicted locations/boxes w.r.t the 8732 prior boxes, a tensor of dimensions (N, 8732, 4) # (N, ?, 5)
        :param predicted_scores: class scores for each of the encoded locations/boxes, a tensor of dimensions (N, 8732, n_classes) # (N, ?, 3)
        :param min_score: minimum threshold for a box to be considered a match for a certain class # 
        :param max_overlap: maximum overlap two boxes can have so that the one with the lower score is not suppressed via NMS
        :param top_k: if there are a lot of resulting detection across all classes, keep only the top 'k'
        :return: detections (boxes, labels, and scores), lists of length batch_size
        """
        batch_size = predicted_locs.size(0)
        n_priors = self.priors_cxcy.size(0)
        predicted_scores = F.softmax(predicted_scores, dim=2)  # (N, 8732, n_classes)

        # Lists to store final predicted boxes, labels, and scores for all images
        all_images_boxes = list()
        all_images_labels = list()
        all_images_scores = list()

        assert n_priors == predicted_locs.size(1) == predicted_scores.size(1)

        for i in range(batch_size):
            # Decode object coordinates from the form we regressed predicted boxes to
            #decoded_locs = cxcy_to_xy(
            #    gcxgcy_to_cxcy(predicted_locs[i], self.priors_cxcy))  # (8732, 4), these are fractional pt. coordinates

            # Lists to store boxes and scores for this image
            image_boxes = list()
            image_labels = list()
            image_scores = list()

            max_scores, best_label = predicted_scores[i].max(dim=1)  # (8732)

            # Check for each class
            for c in range(1, self.n_classes):
                # Keep only predicted boxes and scores where scores for this class are above the minimum score
                class_scores = predicted_scores[i][:, c]  # (8732)
                score_above_min_score = class_scores > min_score  # torch.uint8 (byte) tensor, for indexing
                n_above_min_score = score_above_min_score.sum().item()
                if n_above_min_score == 0:
                    continue
                class_scores = class_scores[score_above_min_score]  # (n_qualified), n_min_score <= 8732
                #class_decoded_locs = decoded_locs[score_above_min_score]  # (n_qualified, 4)
                class_locs = predicted_locs[i][score_above_min_score]
                
                # Sort predicted boxes and scores by scores
                class_scores, sort_ind = class_scores.sort(dim=0, descending=True)  # (n_qualified), (n_min_score)
                #class_decoded_locs = class_decoded_locs[sort_ind]  # (n_min_score, 4)
                class_locs = class_locs[sort_ind]
                
                # Find the overlap between predicted boxes
                print(class_locs)
                print(class_locs.shape)
                overlap = find_jaccard_overlap(class_locs, class_locs)  # (n_qualified, n_min_score)

                # Non-Maximum Suppression (NMS)

                # A torch.uint8 (byte) tensor to keep track of which predicted boxes to suppress
                # 1 implies suppress, 0 implies don't suppress
                suppress = torch.zeros((n_above_min_score), dtype=torch.uint8).to(device)  # (n_qualified)

                # Consider each box in order of decreasing scores
                for box in range(class_decoded_locs.size(0)):
                    # If this box is already marked for suppression
                    if suppress[box] == 1:
                        continue

                    # Suppress boxes whose overlaps (with this box) are greater than maximum overlap
                    # Find such boxes and update suppress indices
                    suppress = torch.max(suppress, overlap[box] > max_overlap)
                    # The max operation retains previously suppressed boxes, like an 'OR' operation

                    # Don't suppress this box, even though it has an overlap of 1 with itself
                    suppress[box] = 0

                # Store only unsuppressed boxes for this class
                image_boxes.append(class_decoded_locs[1 - suppress])
                image_labels.append(torch.LongTensor((1 - suppress).sum().item() * [c]).to(device))
                image_scores.append(class_scores[1 - suppress])

            # If no object in any class is found, store a placeholder for 'background'
            if len(image_boxes) == 0:
                #image_boxes.append(torch.FloatTensor([[0., 0., 1., 1.]]).to(device))
                image_boxes.append(torch.FloatTensor([0.5, 0.5, 1., 1., 0.]).cuda())
                #image_labels.append(torch.LongTensor([0]).to(device))
                image_labels.append(torch.LongTensor([2]).cuda())
                image_scores.append(torch.FloatTensor([0.]).cuda())

            # Concatenate into single tensors
            image_boxes = torch.cat(image_boxes, dim=0)  # (n_objects, 4)
            image_labels = torch.cat(image_labels, dim=0)  # (n_objects)
            image_scores = torch.cat(image_scores, dim=0)  # (n_objects)
            n_objects = image_scores.size(0)

            # Keep only the top k objects
            if n_objects > top_k:
                image_scores, sort_ind = image_scores.sort(dim=0, descending=True)
                image_scores = image_scores[:top_k]  # (top_k)
                image_boxes = image_boxes[sort_ind][:top_k]  # (top_k, 4)
                image_labels = image_labels[sort_ind][:top_k]  # (top_k)

            # Append to lists that store predicted boxes and scores for all images
            all_images_boxes.append(image_boxes)
            all_images_labels.append(image_labels)
            all_images_scores.append(image_scores)

        return all_images_boxes, all_images_labels, all_images_scores  # lists of length batch_size


### Testing

In [139]:
test = SSD300_TEST(2)

In [140]:
print(test.priors_cxcy.size(0))

30558


In [141]:
#predicted_locs, predicted_scores, min_score, max_overlap, top_k
predicted_locs = torch.rand(1, 30558, 5).double() * torch.DoubleTensor([1, 1, .05, .6, -90])
predicted_clas = torch.rand(1, 30558, 3) * 20

all_images_boxes, all_images_labels, all_images_scores = test.detect_objects(predicted_locs, predicted_clas, .5, .5, 10)

tensor([[ 4.9241e-01,  7.1347e-01,  3.1491e-02,  1.1998e-01, -4.6455e+01],
        [ 6.3600e-01,  8.0407e-01,  1.3406e-02,  4.5824e-01, -2.6847e+01],
        [ 6.6440e-01,  5.8237e-01,  3.1617e-02,  3.3151e-01, -7.9109e+01],
        ...,
        [ 2.3737e-01,  4.2522e-01,  1.9739e-02,  3.1821e-01, -4.6757e+01],
        [ 4.2458e-01,  8.8342e-01,  1.9975e-02,  1.5761e-01, -2.3724e+01],
        [ 4.2056e-01,  8.7019e-01,  2.3561e-02,  2.5278e-01, -8.3995e+01]],
       dtype=torch.float64)
torch.Size([539, 5])


NameError: name 'device' is not defined

In [142]:
print(all_images_boxes, all_images_labels, all_images_scores)

[tensor([0.5000, 0.5000, 1.0000, 1.0000, 0.0000], device='cuda:0')] [tensor([2], device='cuda:0')] [tensor([0.], device='cuda:0')]
